In [1]:
import numpy as np
import openpyxl
import json

from analysis.metrics import compute_clip_cohen_d, compute_linear_l

source_file ="/mnt/d/QYQC/PKU/GradPaper/experiments/results.xlsx"

wb = openpyxl.load_workbook(source_file)
# open "human" sub-sheet
ws = wb["human"]
data = []
for row in ws.iter_rows(
    min_row=2, max_row=ws.max_row, min_col=8, max_col=ws.max_column
):
    data.append([cell.value for cell in row])

data=np.array(data)[:,:40]
data

array([[200, 60, 12, ..., 22, 17, 44],
       [1000, 30, 247, ..., 100, 500, 120],
       [1, 0, 1, ..., 0, 1, 1],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None]], dtype=object)

In [2]:
with open("analysis/stat_template.json", "r") as f:
    results = json.load(f)

results[0]["data"]["A"] = data[1]  # task1
results[0]["data"]["B"] = data[0]  # task1
results[1]["data"]["A"] = data[2]  # task2
results[2]["data"]["A"] = data[3]  # task3
results[3]["data"]["A"] = data[4]  # task4
results[4]["data"]["A"] = data[5]  # task5
results[4]["data"]["B"] = data[6]  # task5
results[5]["data"]["A"] = data[7]  # task6.1
results[5]["data"]["B"] = data[9]  # task6.1
results[6]["data"]["A"] = data[8]  # task6.2
results[6]["data"]["B"] = data[5]  # task6.2
results[7]["data"]["A"] = data[10]  # task7.1
results[8]["data"]["A"] = data[11]  # task7.2
results[9]["data"]["A"] = data[12]  # task7.3
results[10]["data"]["B"] = data[13]  # task7.4
results[11]["data"]["A"] = data[14]  # task8
results[11]["data"]["B"] = data[15]  # task8
task9data = np.array([1] * data[16].sum() + [0] * (data[17].sum()))
results[12]["data"]["A"] = task9data.tolist()  # task9

for i in range(len(results) - 1):
    if results[i]["func"] == "d":
        results[i]["result"] = compute_clip_cohen_d(
            results[i]["data"]["A"], results[i]["data"]["B"]
        )

    elif results[i]["func"] == "l":
        results[i]["result"] = compute_linear_l(
            results[i]["data"]["A"],
            target=results[i]["target"],
            morethan=results[i]["morethan"],
        )
    if isinstance(results[i]["data"]["A"], np.ndarray):
        results[i]["data"]["A"] = results[i]["data"]["A"].tolist()
    if isinstance(results[i]["data"]["B"], np.ndarray):
        results[i]["data"]["B"] = results[i]["data"]["B"].tolist()

results[12]["result"] = 2 * (
    data[16].sum() / len(data[16]) - data[17].sum() / len(data[17])
)

dl = [r["result"] for r in results]
e = [
    *(dl[0:5]),
    (dl[5] + dl[6]) / 2,
    (dl[7] + dl[8] + dl[9] + dl[10]) / 4,
    *(dl[11:13]),
]
print(e)

with open("analysis/stat_results/human.json", "w") as f:
    json.dump(
        {
            "overview": {
                "name": "human",
                "dl": dl,
                "e": e,
            },
            "details": results,
        },
        f,
        indent=4,
    )

[0.9946268563170335, 0.9, 1.1, 1.0, 0.42118727507167525, 0.3969881820729002, 0.8015000497740514, 0.8412482325527576, 0.55]
